<a href="https://colab.research.google.com/github/vasiliyeskin/MachineLearningExperiences/blob/master/FashionMNIST/fashion_mnist_keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Оптимизация гиперпараметров нейросети с помощью [Keras Tuner](https://github.com/keras-team/keras-tuner)

Взято с https://www.youtube.com/watch?v=M08yHFPKTNI&list=PLtPJ9lKvJ4oiz9aaL_xcZd-x0qd8G0VN_&index=11.

Чтобы запускать и редактировать код, сохраните копию этого ноутбука себе (File->Save a copy in Drive...). Свою копию вы сможете изменять и запускать.

Не забудьте подключить GPU, чтобы сеть обучалась быстрее (Runtime -> Change Runtime Type -> Hardware Accelerator -> GPU).



## Гиперпараметры обучения нейронной сети

- Количество слоев нейронной сети
- Количество нейронов в каждом слое
- Функции активации, которые используются в слоях
- Тип оптимизатора при обучении нейронной сети
- Количество эпох обучения

## Установка Keras Tuner

In [ ]:
pip install -U keras-tuner

Requirement already up-to-date: keras-tuner in /usr/local/lib/python3.6/dist-packages (1.0.1)


## Подключаем нужные пакеты

In [ ]:
%tensorflow_version 2.x
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import utils
from google.colab import files
from kerastuner.tuners import RandomSearch, Hyperband, BayesianOptimization
import numpy as np

TensorFlow 2.x selected.


## Подготовка данных для обучения сети

In [ ]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train / 255 
x_test = x_test / 255 
y_train = utils.to_categorical(y_train, 10)
y_test = utils.to_categorical(y_test, 10)

## Задаем функцию создания нейронной сети

In [ ]:
def build_model(hp):
    model = Sequential()
    activation_choice = hp.Choice('activation', values=['relu', 'sigmoid', 'tanh', 'elu', 'selu'])    
    model.add(Dense(units=hp.Int('units_input',    # Полносвязный слой с разным количеством нейронов
                                   min_value=512,    # минимальное количество нейронов - 128
                                   max_value=1024,   # максимальное количество - 1024
                                   step=32),
                    input_dim=784,
                    activation=activation_choice))
    model.add(Dense(units=hp.Int('units_hidden',        
                                   min_value=128,   
                                   max_value=600,   
                                   step=32),
                    activation=activation_choice))   
    model.add(Dense(10, activation='softmax'))
    model.compile(
        optimizer=hp.Choice('optimizer', values=['adam','rmsprop','SGD']),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

## Создаем tuner

Доступные типы тюнеров: 
- RandomSearch - случайный поиск.
- Hyperband - алгоритм оптимизации на основе многорукого бандита, Li, Lisha, and Kevin Jamieson. ["Hyperband: A Novel Bandit-Based Approach to Hyperparameter Optimization."Journal of Machine Learning Research 18 (2018): 1-52](http://jmlr.org/papers/v18/16-558.html).
- BayesianOptimization - [байесовская оптимизация](https://en.wikipedia.org/wiki/Bayesian_optimization).

In [ ]:
tuner = RandomSearch(
    build_model,                 # функция создания модели
    objective='val_accuracy',    # метрика, которую нужно оптимизировать - 
                                 # доля правильных ответов на проверочном наборе данных
    max_trials=80,               # максимальное количество запусков обучения 
    directory='test_directory'   # каталог, куда сохраняются обученные сети  
    )

## Запускаем подбор гиперпараметров

Пространство поиска

In [ ]:
tuner.search_space_summary()

Подбор гиперпараметров

In [ ]:
tuner.search(x_train,                  # Данные для обучения
             y_train,                  # Правильные ответы
             batch_size=256,           # Размер мини-выборки
             epochs=20,                # Количество эпох обучения 
             validation_split=0.2,     # Часть данных, которая будет использоваться для проверки
             )

Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 3s 68us/sample - loss: 0.6393 - accuracy: 0.7679 - val_loss: 0.4416 - val_accuracy: 0.8418
Epoch 2/20
48000/48000 [==============================] - 1s 21us/sample - loss: 0.4150 - accuracy: 0.8459 - val_loss: 0.3811 - val_accuracy: 0.8628
Epoch 3/20
48000/48000 [==============================] - 1s 22us/sample - loss: 0.3632 - accuracy: 0.8643 - val_loss: 0.3496 - val_accuracy: 0.8755
Epoch 4/20
48000/48000 [==============================] - 1s 21us/sample - loss: 0.3296 - accuracy: 0.8760 - val_loss: 0.4933 - val_accuracy: 0.8196
Epoch 5/20
48000/48000 [==============================] - 1s 21us/sample - loss: 0.3034 - accuracy: 0.8850 - val_loss: 0.3554 - val_accuracy: 0.8677
Epoch 6/20
48000/48000 [==============================] - 1s 22us/sample - loss: 0.2858 - accuracy: 0.8917 - val_loss: 0.3314 - val_accuracy: 0.8823
Epoch 7/20
48000/48000 [==============================] 

Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 1s 29us/sample - loss: 0.7928 - accuracy: 0.7337 - val_loss: 0.5612 - val_accuracy: 0.7797
Epoch 2/20
48000/48000 [==============================] - 1s 22us/sample - loss: 0.4829 - accuracy: 0.8243 - val_loss: 0.4492 - val_accuracy: 0.8372
Epoch 3/20
48000/48000 [==============================] - 1s 22us/sample - loss: 0.4288 - accuracy: 0.8435 - val_loss: 0.4463 - val_accuracy: 0.8371
Epoch 4/20
48000/48000 [==============================] - 1s 22us/sample - loss: 0.3998 - accuracy: 0.8551 - val_loss: 0.4435 - val_accuracy: 0.8342
Epoch 5/20
48000/48000 [==============================] - 1s 22us/sample - loss: 0.3795 - accuracy: 0.8615 - val_loss: 0.3872 - val_accuracy: 0.8603
Epoch 6/20
48000/48000 [==============================] - 1s 22us/sample - loss: 0.3625 - accuracy: 0.8660 - val_loss: 0.4199 - val_accuracy: 0.8396
Epoch 7/20
48000/48000 [==============================] 

Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 2s 33us/sample - loss: 0.8770 - accuracy: 0.7216 - val_loss: 0.6018 - val_accuracy: 0.7872
Epoch 2/20
48000/48000 [==============================] - 1s 23us/sample - loss: 0.5098 - accuracy: 0.8116 - val_loss: 0.4332 - val_accuracy: 0.8406
Epoch 3/20
48000/48000 [==============================] - 1s 24us/sample - loss: 0.4270 - accuracy: 0.8396 - val_loss: 0.4261 - val_accuracy: 0.8353
Epoch 4/20
48000/48000 [==============================] - 1s 25us/sample - loss: 0.3794 - accuracy: 0.8570 - val_loss: 0.5458 - val_accuracy: 0.7934
Epoch 5/20
48000/48000 [==============================] - 1s 25us/sample - loss: 0.3501 - accuracy: 0.8679 - val_loss: 0.4572 - val_accuracy: 0.8296
Epoch 6/20
48000/48000 [==============================] - 1s 22us/sample - loss: 0.3295 - accuracy: 0.8749 - val_loss: 0.3596 - val_accuracy: 0.8705
Epoch 7/20
48000/48000 [==============================] 

Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 1s 30us/sample - loss: 0.6634 - accuracy: 0.7602 - val_loss: 0.4849 - val_accuracy: 0.8262
Epoch 2/20
48000/48000 [==============================] - 1s 24us/sample - loss: 0.4228 - accuracy: 0.8441 - val_loss: 0.4226 - val_accuracy: 0.8443
Epoch 3/20
48000/48000 [==============================] - 1s 22us/sample - loss: 0.3620 - accuracy: 0.8648 - val_loss: 0.4347 - val_accuracy: 0.8419
Epoch 4/20
48000/48000 [==============================] - 1s 23us/sample - loss: 0.3266 - accuracy: 0.8788 - val_loss: 0.3910 - val_accuracy: 0.8592
Epoch 5/20
48000/48000 [==============================] - 1s 22us/sample - loss: 0.3043 - accuracy: 0.8860 - val_loss: 0.3357 - val_accuracy: 0.8833
Epoch 6/20
48000/48000 [==============================] - 1s 22us/sample - loss: 0.2840 - accuracy: 0.8914 - val_loss: 0.3380 - val_accuracy: 0.8781
Epoch 7/20
48000/48000 [==============================] 

Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 1s 31us/sample - loss: 0.7598 - accuracy: 0.7378 - val_loss: 0.4825 - val_accuracy: 0.8259
Epoch 2/20
48000/48000 [==============================] - 1s 23us/sample - loss: 0.4605 - accuracy: 0.8261 - val_loss: 0.3831 - val_accuracy: 0.8601
Epoch 3/20
48000/48000 [==============================] - 1s 23us/sample - loss: 0.3934 - accuracy: 0.8538 - val_loss: 0.4124 - val_accuracy: 0.8565
Epoch 4/20
48000/48000 [==============================] - 1s 22us/sample - loss: 0.3577 - accuracy: 0.8653 - val_loss: 0.3613 - val_accuracy: 0.8642
Epoch 5/20
48000/48000 [==============================] - 1s 22us/sample - loss: 0.3332 - accuracy: 0.8741 - val_loss: 0.3737 - val_accuracy: 0.8576
Epoch 6/20
48000/48000 [==============================] - 1s 22us/sample - loss: 0.3122 - accuracy: 0.8838 - val_loss: 0.3705 - val_accuracy: 0.8612
Epoch 7/20
48000/48000 [==============================] 

...

Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 1s 29us/sample - loss: 0.5732 - accuracy: 0.7991 - val_loss: 0.4495 - val_accuracy: 0.8375
Epoch 2/20
48000/48000 [==============================] - 1s 22us/sample - loss: 0.4121 - accuracy: 0.8521 - val_loss: 0.3919 - val_accuracy: 0.8575
Epoch 3/20
48000/48000 [==============================] - 1s 22us/sample - loss: 0.3695 - accuracy: 0.8643 - val_loss: 0.3805 - val_accuracy: 0.8642
Epoch 4/20
48000/48000 [==============================] - 1s 23us/sample - loss: 0.3423 - accuracy: 0.8739 - val_loss: 0.3569 - val_accuracy: 0.8703
Epoch 5/20
48000/48000 [==============================] - 1s 22us/sample - loss: 0.3193 - accuracy: 0.8809 - val_loss: 0.3850 - val_accuracy: 0.8629
Epoch 6/20
48000/48000 [==============================] - 1s 23us/sample - loss: 0.3012 - accuracy: 0.8890 - val_loss: 0.3467 - val_accuracy: 0.8737
Epoch 7/20
48000/48000 [==============================] 

Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 1s 28us/sample - loss: 1.0537 - accuracy: 0.6715 - val_loss: 0.7450 - val_accuracy: 0.7619
Epoch 2/20
48000/48000 [==============================] - 1s 22us/sample - loss: 0.6825 - accuracy: 0.7807 - val_loss: 0.6254 - val_accuracy: 0.7959
Epoch 3/20
48000/48000 [==============================] - 1s 22us/sample - loss: 0.5997 - accuracy: 0.8046 - val_loss: 0.5704 - val_accuracy: 0.8090
Epoch 4/20
48000/48000 [==============================] - 1s 21us/sample - loss: 0.5557 - accuracy: 0.8169 - val_loss: 0.5431 - val_accuracy: 0.8168
Epoch 5/20
48000/48000 [==============================] - 1s 22us/sample - loss: 0.5277 - accuracy: 0.8244 - val_loss: 0.5204 - val_accuracy: 0.8223
Epoch 6/20
48000/48000 [==============================] - 1s 22us/sample - loss: 0.5078 - accuracy: 0.8290 - val_loss: 0.5065 - val_accuracy: 0.8246
Epoch 7/20
48000/48000 [==============================] 

Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 1s 27us/sample - loss: 0.8235 - accuracy: 0.7287 - val_loss: 0.6066 - val_accuracy: 0.7950
Epoch 2/20
48000/48000 [==============================] - 1s 21us/sample - loss: 0.5679 - accuracy: 0.8091 - val_loss: 0.5390 - val_accuracy: 0.8145
Epoch 3/20
48000/48000 [==============================] - 1s 22us/sample - loss: 0.5133 - accuracy: 0.8246 - val_loss: 0.5031 - val_accuracy: 0.8224
Epoch 4/20
48000/48000 [==============================] - 1s 21us/sample - loss: 0.4849 - accuracy: 0.8329 - val_loss: 0.4877 - val_accuracy: 0.8265
Epoch 5/20
48000/48000 [==============================] - 1s 23us/sample - loss: 0.4652 - accuracy: 0.8391 - val_loss: 0.4692 - val_accuracy: 0.8320
Epoch 6/20
48000/48000 [==============================] - 1s 22us/sample - loss: 0.4514 - accuracy: 0.8431 - val_loss: 0.4647 - val_accuracy: 0.8338
Epoch 7/20
48000/48000 [==============================] 

Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 1s 31us/sample - loss: 0.7107 - accuracy: 0.7437 - val_loss: 0.6047 - val_accuracy: 0.7742
Epoch 2/20
48000/48000 [==============================] - 1s 22us/sample - loss: 0.4537 - accuracy: 0.8280 - val_loss: 0.4758 - val_accuracy: 0.8216
Epoch 3/20
48000/48000 [==============================] - 1s 23us/sample - loss: 0.3961 - accuracy: 0.8517 - val_loss: 0.4363 - val_accuracy: 0.8377
Epoch 4/20
48000/48000 [==============================] - 1s 22us/sample - loss: 0.3573 - accuracy: 0.8665 - val_loss: 0.4420 - val_accuracy: 0.8358
Epoch 5/20
48000/48000 [==============================] - 1s 25us/sample - loss: 0.3302 - accuracy: 0.8758 - val_loss: 0.4090 - val_accuracy: 0.8448
Epoch 6/20
48000/48000 [==============================] - 1s 25us/sample - loss: 0.3101 - accuracy: 0.8832 - val_loss: 0.4227 - val_accuracy: 0.8428
Epoch 7/20
48000/48000 [==============================] 

INFO:tensorflow:Oracle triggered exit


## Выбираем лучшую модель

In [ ]:
tuner.results_summary()

Получаем три лучших модели

In [ ]:
models = tuner.get_best_models(num_models=3)

Оцениваем качество модели на тестовых данных

In [ ]:
for model in models:
  model.summary()
  model.evaluate(x_test, y_test)
  print() 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 896)               703360    
_________________________________________________________________
dense_1 (Dense)              (None, 448)               401856    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                4490      
Total params: 1,109,706
Trainable params: 1,109,706
Non-trainable params: 0
_________________________________________________________________
10000/10000 [==============================] - 1s 92us/sample - loss: 0.3721 - accuracy: 0.8941

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 608)               477280    
_____________________________________________________________